In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
data_folder = "data/"
filename = "customBenchmark_kepler_query_TwoNodes.json"

In [ ]:
file = open(data_folder + filename)
data = file.read()
data = json.loads(data)

In [ ]:
data.keys()

In [ ]:
result = {}

for metric in data:
    temp = []
    if metric == 'node_cpu_seconds_total':
        for node_metric in data[metric]:
            if node_metric['metric']['mode'] == "idle":
                continue
            tmp = [[node_metric['metric']['instance'], node_metric['metric']['cpu'], values[0], values[1]] for values in node_metric['values']]
            tmp_df = pd.DataFrame(tmp, columns =['node', 'cpu', 'time', 'value'])
            temp.append(tmp_df)
    elif metric == 'kepler_node_core_joules_total':
        for node_metric in data[metric]:
            tmp = [[node_metric['metric']['instance'], node_metric['metric']['package'], values[0], values[1]] for values in node_metric['values']]
            tmp_df = pd.DataFrame(tmp, columns =['node', 'cpu', 'time', 'value'])
            temp.append(tmp_df)
    elif metric == 'kepler_node_dram_joules_total':
        for node_metric in data[metric]:
            tmp = [[node_metric['metric']['instance'], node_metric['metric']['package'], values[0], values[1]] for values in node_metric['values']]
            tmp_df = pd.DataFrame(tmp, columns =['node', 'cpu', 'time', 'value'])
            temp.append(tmp_df)
    elif metric == 'kepler_node_package_joules_total':
        for node_metric in data[metric]:
            tmp = [[node_metric['metric']['instance'], node_metric['metric']['package'], values[0], values[1]] for values in node_metric['values']]
            tmp_df = pd.DataFrame(tmp, columns =['node', 'cpu', 'time', 'value'])
            temp.append(tmp_df)
    else:
        continue
    t = pd.concat(temp)
    t['value'] = t['value'].astype(float)
    result[metric] = t.reset_index(drop = True)

In [ ]:
cpu_util = result['node_cpu_seconds_total']
cpu_util_agg = cpu_util.groupby(['node', 'time'], as_index=False)['value'].sum()
node_core_power = result['kepler_node_core_joules_total']
node_core_power = node_core_power.groupby(['node', 'time'], as_index=False)['value'].sum()
node_dram_power = result['kepler_node_dram_joules_total']
node_dram_power = node_dram_power.groupby(['node', 'time'], as_index=False)['value'].sum()
node_package_power = result['kepler_node_package_joules_total']
node_package_power = node_package_power.groupby(['node', 'time'], as_index=False)['value'].sum()

In [ ]:
final = pd.merge(cpu_util_agg, node_core_power, on = ['time', 'node'])
final.rename(columns={'value_x':'util', 'value_y':'core'}, inplace=True)
final.sort_values('time', inplace=True)

final = pd.merge(final, node_dram_power, on = ['time', 'node'])
final.rename(columns={'value':'dram'}, inplace=True)

final = pd.merge(final, node_package_power, on = ['time', 'node'])
final.rename(columns={'value':'package'}, inplace=True)

In [ ]:
final

In [ ]:
import model

In [ ]:
m = model.UtilisationPowerModel()
node_power_models = {}
for node in final['node'].unique():
#     if node != '10.242.64.5':
#         continue
    node_df = final[final['node'] == node]
    node_df['util'] = node_df['util'].diff()
    node_df['core'] = node_df['core'].diff()
    node_df['dram'] = node_df['dram'].diff()
    node_df['package'] = node_df['package'].diff()
    node_df['energy'] = node_df['core'] + node_df['dram'] + node_df['package']
    node_df.fillna(0, inplace=True)
    node_df['energy'] = node_df['energy'].shift(1)
    node_df = node_df[(node_df[['util']] != 0).all(axis=1)]
    node_df = node_df[(node_df[['energy']] != 0).all(axis=1)]
    max_util = max(node_df['util'])
    node_df['util'] = node_df['util']/max_util
    node_df['time_diff'] = node_df['time'].diff()
    node_df['power'] = node_df['energy']/node_df['time_diff']
    node_df['power'].fillna(node_df['power'].mean(), inplace=True)
    node_power_models[node] = m.get_model(node_df[['util', 'power']])

In [ ]:
node_power_models